In [1]:
import requests
from selenium.webdriver import (Chrome, Firefox, ChromeOptions, FirefoxProfile)
import pymongo
import datetime
import yaml
import time
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [21]:
class FBWebScraper():

    def __init__(self, my_email, my_password, my_profile_url, statuses=50, scroll_time=7, browser='Chrome'):

        self.my_email = my_email
        self.my_password = my_password
        self.my_profile_url = my_profile_url

        self.number_of_statuses = statuses
        self.scroll_time = scroll_time

        # Initialize MongoDB
       
        self.mc = pymongo.MongoClient()
        self.db = self.mc['pprediction']    #database name
        self.statuses = self.db['statuses']   #collection name

        self.set_browser(browser)

        person_dict = self.statuses.find_one({'friends_dict': {'$exists': True}})
        if person_dict == None:
            self.friends_dict = {}
        else:
            self.friends_dict = person_dict['friends_dict']

    # Sets the browser to scrape with
    def set_browser(self, browser):
        # CHROME
        if browser == 'Chrome':
            options = ChromeOptions();
            options.add_argument("--disable-notifications");
            self.browser = Chrome(options=options)

        # FIREFOX
        if browser == 'Firefox':
            profile = FirefoxProfile();
            profile.set_preference("dom.webnotifications.enabled", False);
            self.browser = Firefox(firefox_profile=profile)

    # Opens facebook in the browser
    def open_fb(self):
        # Login to FB in Selenium Browser
        url = 'https://www.facebook.com/'
        self.browser.get(url)

        email = self.browser.find_element_by_id('email')
        password = self.browser.find_element_by_id('pass')

        email.send_keys(self.my_email)
        password.send_keys(self.my_password)
       
        time.sleep(10)
        self.browser.find_element_by_xpath('/html/body/div[1]/div[2]/div[1]/div/div/div/div[2]/div/div[1]/form/div[2]/button').click()
        self.browser.maximize_window()
       
    # Creates a dictionary of friends and their profile links, where key=profile_url and value=friends_name
    def create_friends_dict(self):
        
        #self.browser.find_element_by_xpath('/html/body/div[10]/div[1]/div/div[2]/div/div/div/div/div/div[1]/div/div[3]/div/div[1]/div[1]/div/div[1]').click()
        
        #arrow
        self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[4]/div[1]/span/div/div[1]/i').click()
        #self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[4]/div[1]/span/div/div[1]/img').click()
        time.sleep(5)
       
        #view profile
        #self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[4]/div[2]/div/div[2]/div[1]/div[1]/div/div/div/div/div/div/div/div/div[1]/div/div[1]/a/div[1]/div[2]/div').click()
        self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[4]/div[2]/div/div/div[1]/div[1]/div/div/div/div/div/div/div/div/div[1]/div/div[1]/a/div[1]').click()
       
        time.sleep(10)
       
        #friends

        #self.browser.switch_to.frame("iframe")
        time.sleep(10)
        self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[3]/div/div/div/div[1]/div/div/div[1]/div/div/div/div/div/div/a[3]/div[1]').click()
        
        time.sleep(self.scroll_time)

        #self.number_of_friends = 5
        time.sleep(3)
        # Get scroll height
        last_height = self.browser.execute_script("return document.body.scrollHeight")
       
        
        dict1={}
        x=0
        self.statuses.delete_many({})
        for i in range(5):
            html = self.browser.page_source
            x+=1
            time.sleep(3)
            self.browser.execute_script("window.scrollTo(0, 370)")
            xpath = "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div/div/div/div/div/div/div/div/div[3]/div["+str(x)+"]"
            
            time.sleep(5)
            #open profile
            self.browser.find_element_by_xpath(xpath).click()
            #extract statuses
            time.sleep(15)
            
            #self.name= self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div[2]/div/div/div/div[1]/div/div/span/h1').text
            #self.name= self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div[2]/div/div[2]/div[3]/div[1]/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div').text
            self.name= self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div[3]/div/div/div[1]/div/div/span/h1').text
            print(self.name)
            
#             #self.browser.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div[1]/div/div/div/a').click()
            
            
#             time.sleep(5)
            
#             #self.profile_pic_url = self.browser.get_screenshot_as_png()
#             self.browser.save_screenshot('C:/Users/home/Desktop/trial/public/profile_pics/'+self.name+'.png')
#             #im.save('C:/Users/HP/Desktop/copy/facebook/profile_pics/'+self.name+'.png')
            
            
#             time.sleep(2)
            
#             from PIL import Image
#             from io import BytesIO
#             self.browser.back()
            
#             image_obj = Image.open('C:/Users/home/Desktop/trial/public/profile_pics/'+self.name+'.png')
#             cropped_image = image_obj.crop((190,0,775,590))
#             cropped_image.save('C:/Users/home/Desktop/trial/public/profile_pics/'+self.name+'.png')
#             cropped_image.show()
            
#             basewidth = 500
            
#             wpercent = (basewidth / float(cropped_image.size[0]))
#             hsize = int((float(cropped_image.size[1]) * float(wpercent)))
#             cropped_image = cropped_image.resize((basewidth, hsize), Image.ANTIALIAS)
#             cropped_image.save('C:/Users/home/Desktop/trial/static/profile_pics'+self.name+'.png')
#             cropped_image.show()
            
            #im.save('C:/Users/HP/Desktop/copy/facebook/profile_pics/'+self.name+'.png')
            self.profile_pic_url = 'profile_pics/'+self.name+'.png'

            xp1 = "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div[1]/div[1]/div/div/div[4]/div[2]/div/div[2]/div[3]/div[1]/div/div/div/div/div/div/div/div/div/div/div[2]/div/div[3]/div/div/div/div"
            self.status = self.browser.find_element_by_xpath(xp1).text
            
            print(self.status)
            
            self.statuses.insert_one({
            'name': self.name,
            'statuses': self.status,
            'profile_pic_url': self.profile_pic_url
              }
            )
            
            self.browser.back()
            time.sleep(7)
        self.browser.quit()            
        
          
        html = self.browser.page_source
        

In [ ]:
if __name__ == '__main__':
    with open('fb_login_creds.yaml', 'r') as stream:
        try:
            y = yaml.load(stream)
            my_password = y['password']
            my_email = y['email']
            my_profile_url = y['profile_url']
        except yaml.YAMLError as exc:
            print(exc)

    FBWS = FBWebScraper(
        my_email=my_email,
        my_password=my_password,
        my_profile_url=my_profile_url,
        browser='Chrome'
    )
 
    FBWS.open_fb()
    if FBWS.friends_dict == {}:
        FBWS.create_friends_dict()
    #FBWS.scrape_friends_statuses()

<ipython-input-22-eda5d1c1ecc7>:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  y = yaml.load(stream)


Tejas Ghute
Where the life takes you is not your destiny
It's your choice
Mrunmayee Pable
If life was predictable it would cease to be life, and be without flavor!
Jeet Choudhari
You will face many defeats in life, but never let yourself be defeated.
Mansi Pable
Life is all about becoming a better version of yourself!
